In [1]:
import pickle
import numpy as np
from dataset import Dataset
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

In [3]:
dataset = Dataset(sensor_data, labels, interp_funcs)

In [4]:
def plot_data(X, time_arr, title):
    data_T = X.T
    fig = go.Figure()

    for i, data in enumerate(data_T):
        fig.add_trace(go.Scatter(x=time_arr,
                                 y=data,
                                 mode="markers",
                                 name=f"H.S. {i}"))

    fig.update_layout(title=title, title_x=0.5, width=1000, height=400)
    fig.update_traces(marker=dict(size=2))
    fig.show()

In [ ]:
for s in range(0, 8, 2):
    matrix = 0
    s_l = s
    s_r = s+1
    X, y, time_arr, targets = dataset.get_sensor_pair_cls(matrix,
                                                          (s_l, s_r),
                                                          num_samples=100,
                                                          as_log=True,
                                                          as_mean=False)

    title = f"Matrix {matrix}, Sensors {s_l} - {s_r}"
    # plot_data(X, time_arr, title)

    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.2,
                                                        random_state=42)

    clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    print(f"Dec Tree Accuracy: {accuracy:.3f} F1: {f1:.3f}")

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(max_iter=5000)
    clf = clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    print(f"MLP: Accuracy: {accuracy:.3f} F1: {f1:.3f}")

In [5]:
def plot_data_pair(X_l, X_r, time_arr, title):
    data_T_l = X_l.T
    data_T_r = X_r.T
    fig = go.Figure()

    for i, data in enumerate(data_T_l):
        fig.add_trace(go.Scatter(x=time_arr,
                                 y=data,
                                 mode="markers",
                                 name=f"L {i}"))
    for i, data in enumerate(data_T_r):
        fig.add_trace(go.Scatter(x=time_arr,
                                 y=data,
                                 mode="markers",
                                 name=f"R {i}"))

    fig.update_layout(title=title, title_x=0.5, width=1000, height=600)
    fig.update_traces(marker=dict(size=2))
    fig.show()

In [ ]:
matrix = 0
for s in range(0, 2, 2):
    s_l = s
    s_r = s+1
    X_l, y_l, time_arr_l, targets_l = dataset.get_sensor_cls(matrix,
                                                             s_l,
                                                             num_samples=100,
                                                             as_log=True)
    X_r, y_r, time_arr_r, targets_r = dataset.get_sensor_cls(matrix,
                                                             s_r,
                                                             num_samples=100,
                                                             as_log=True)
    title = f"Matrix {matrix}, Sensors {s_l} (L) - {s_r} (R)"

    plot_data_pair(X_l, X_r, time_arr_l, title)

In [7]:
X, y, time_arr, targets = dataset.get_sensor_pair_cls(0,
                                                      (0, 1),
                                                      num_samples=100,
                                                      as_log=True,
                                                      as_mean=False)

X_clean, y_clean = dataset.clean_up_regression_data(X, targets)